In [1]:
from townsnet import Region

regions_dict = {
  1: 'Ленинградская область',
  3138: 'Санкт-Петербург',
  3268: 'Москва',
  3427: 'Волгоградская область',
  3902: 'Тульская область',
  4013: 'Омская область',
  4437: 'Краснодарский край',
  4882: 'Тюменская область',
  5188: 'Московская область',
}

regions = {region_id:Region.from_pickle(f'data/regions/{region_id}/{region_id}.pickle') for region_id in regions_dict.keys()}

In [2]:
for region_id, region in regions.items():
  print(region_id, len(region.get_services_gdf()))

1 26602
3138 55892
3268 109623
3427 14784
3902 11545
4013 11951
4437 51867
4882 15139
5188 97400


In [3]:
from townsnet import Provision
from tqdm import tqdm
from multiprocessing import Pool

for region_id, region in tqdm(regions.items()):
  provision = Provision(region=region)
  
  def calculate_provision(st):
    data_path = f'data/regions/{region_id}/provisions/'
    d_gdf, s_gdf, t_gdf, l_gdf = provision.calculate(st)
    t_gdf['settlement_name'] = t_gdf['settlement_name'].apply(lambda v : None if v==0 else v)
    t_gdf['district_name'] = t_gdf['district_name'].apply(lambda v : None if v==0 else v)
    d_gdf.to_parquet(data_path + f'{st.name}_districts.parquet')
    s_gdf.to_parquet(data_path + f'{st.name}_settlements.parquet')
    t_gdf.to_parquet(data_path + f'{st.name}_towns.parquet')
    l_gdf.to_parquet(data_path + f'{st.name}_links.parquet')
  
  with Pool() as pool:
    pool.map(calculate_provision, region.service_types)

100%|██████████| 9/9 [05:13<00:00, 34.83s/it]


In [4]:
import geopandas as gpd
import pandas as pd

source = 'https://www.openstreetmap.org/'
period = '2023'

for region_id, region_name in regions_dict.items():
  data_path = f'data/regions/{region_id}/'
  region = regions[region_id]
  st_df = region.get_service_types_df()
  values = []
  for i,infrastructure in enumerate(st_df.infrastructure.unique()):
    st_sub_df = st_df[st_df.infrastructure == infrastructure]
    for j, st_name in enumerate(st_sub_df.name):
      code = f'5.{i+1}.{j+1}'
      d_gdf = gpd.read_parquet(data_path + f'provisions/{st_name}_districts.parquet')
      s_gdf = gpd.read_parquet(data_path + f'provisions/{st_name}_settlements.parquet')
      t_gdf = gpd.read_parquet(data_path + f'provisions/{st_name}_towns.parquet')
      l_gdf = gpd.read_parquet(data_path + f'provisions/{st_name}_links.parquet')
      total = Provision.total_provision(d_gdf)
      values.append({
        '№ п/п': code,
        'Значение': round(total,2) if total is not None else None,
        'Территория': region_name,
        'Источник': source,
        'Период': period,
      })
      for district_id in d_gdf.index:
        district_name = d_gdf.loc[district_id, 'name']
        total = d_gdf.loc[district_id, 'provision']
        values.append({
          '№ п/п': code,
          'Значение': round(total,2) if total is not None else None,
          'Территория': district_name,
          'Источник': source,
          'Период': period,
        })
      for settlement_id in s_gdf.index:
        settlement_name = s_gdf.loc[settlement_id, 'name']
        total = s_gdf.loc[settlement_id, 'provision']
        values.append({
          '№ п/п': code,
          'Значение': round(total,2) if total is not None else None,
          'Территория': settlement_name,
          'Источник': source,
          'Период': period,
        })
  values_df = pd.DataFrame(values)
  values_df.to_excel(data_path+f'{region_id} {region_name}.xlsx', index=False)

In [5]:
region = regions[1]

In [6]:
from shapely import set_precision
from statistics import mean

def get_region_provisions(region, region_id):
  data_path = f'data/regions/{region_id}/provisions/'
  d_gdfs = []
  s_gdfs = []
  t_gdfs = []
  for st in region.service_types:
    prov_col = f'provision_{st.name}'

    d_gdf = gpd.read_parquet(data_path + f'{st.name}_districts.parquet')[['geometry', 'name', 'provision']].rename(columns={
      'provision': f'provision_{st.name}',
      'name': 'district_name'
    })
    d_gdf[prov_col] = d_gdf[prov_col].apply(lambda p : round(p,2))
    d_gdfs.append(d_gdf)

    s_gdf = gpd.read_parquet(data_path + f'{st.name}_settlements.parquet')[['geometry', 'name', 'provision']].rename(columns={
      'provision': f'provision_{st.name}',                                                                                                                               
      'name': 'settlement_name'
    })
    s_gdf[prov_col] = s_gdf[prov_col].apply(lambda p : round(p,2))
    s_gdfs.append(s_gdf)

    t_gdf = gpd.read_parquet(data_path + f'{st.name}_towns.parquet')[['geometry', 'town_name', 'provision']].rename(columns={'provision': f'provision_{st.name}'})
    t_gdf[prov_col] = t_gdf[prov_col].apply(lambda p : round(p,2))
    t_gdfs.append(t_gdf)

  d_gdf = d_gdfs[0][['geometry', 'district_name']]
  s_gdf = s_gdfs[0][['geometry', 'settlement_name']]
  t_gdf = t_gdfs[0][['geometry', 'town_name']]

  d_gdf = pd.concat([d_gdf, *[gdf[filter(lambda c : 'provision' in c, gdf.columns)] for gdf in d_gdfs]], axis=1)
  s_gdf = pd.concat([s_gdf, *[gdf[filter(lambda c : 'provision' in c, gdf.columns)] for gdf in s_gdfs]], axis=1)
  t_gdf = pd.concat([t_gdf, *[gdf[filter(lambda c : 'provision' in c, gdf.columns)] for gdf in t_gdfs]], axis=1)

  for gdf in [d_gdf, s_gdf, t_gdf]:
    gdf['provision'] = gdf[filter(lambda c : 'provision' in c, gdf.columns)].apply(mean, axis=1).apply(lambda p : round(p, 2))
    gdf.to_crs(4326, inplace=True)
    gdf['geometry'] = set_precision(gdf['geometry'], grid_size=0.0001)

  return d_gdf, s_gdf, t_gdf

In [7]:
for region_id, region in regions.items():
  data_path = f'data/regions/{region_id}/provisions/'
  d_gdf, s_gdf, t_gdf = get_region_provisions(region, region_id)
  d_gdf.to_file(data_path + 'districts.geojson')
  s_gdf.to_file(data_path + 'settlements.geojson')
  t_gdf.to_file(data_path + 'towns.geojson')

In [8]:
region = Region.from_pickle('data/regions/1/1.pickle')
region.settlements

,geometry,name
0,"MULTIPOLYGON (((580049.515 6617067.720, 580225...",Самойловское сельское поселение
1,"MULTIPOLYGON (((573514.444 6620389.851, 573579...",Большедворское сельское поселение
2,"MULTIPOLYGON (((564707.328 6594903.594, 564341...",Пикалевское городское поселение
3,"MULTIPOLYGON (((560629.509 6557332.762, 560642...",Борское сельское поселение
4,"MULTIPOLYGON (((560659.317 6585430.874, 559919...",Бокситогорское городское поселение
...,...,...
183,"MULTIPOLYGON (((360431.648 6619592.469, 360583...",Фёдоровское городское поселение
184,"MULTIPOLYGON (((387941.398 6612286.590, 388009...",Нурминское сельское поселение
185,"MULTIPOLYGON (((363803.737 6614904.469, 363821...",Красноборское городское поселение
186,"MULTIPOLYGON (((392924.072 6551507.217, 393174...",Трубникоборское сельское поселение
